In [70]:
from datetime import datetime, timezone
from typing import List
from pynwb import NWBFile, NWBHDF5IO
from pynwb.file import Subject
from pynwb.ecephys import ElectricalSeries, LFP
import nixio
from os.path import join
import numpy as np
import pandas as pd
from dataclasses import dataclass

In [71]:
def main():
    context = create_context(1, 1)
    write_subject(context)

In [72]:
@dataclass(frozen=True)
class Context:
    subject: int
    session: int
    nix: nixio.File
    nwb: NWBFile

In [73]:
def create_context(subject: int, session: int) -> Context:
    nix = _read_nix(subject, session)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description=_get_task(nix),
        identifier=f"Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject{subject:02}_session{session:02}",
        session_start_time=datetime(2019, 3, 27, 12, 00, tzinfo=timezone.utc),  # TODO: Get this from Johannes
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        related_publications=_get_related_publications(nix)
    )
    return Context(subject, session, nix, nwb)


In [74]:
def _read_nix(subject: int, session: int) -> nixio.File:
    _nix_dir = join("/", "mnt", "c", "Users", "conta", "git", "USZ_NCH", "Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM",
                    "data_nix")
    _file_path = join(_nix_dir, f"Data_Subject_{subject:02}_Session_{session:02}.h5")
    return nixio.File.open(_file_path, nixio.FileMode.ReadOnly)

In [75]:
def _get_task(nix: nixio.File) -> str:
    task = nix.sections["Task"].props
    task_name = task["Task name"].values[0]
    # Broken UTF-8 in file
    task_desc = "The task is a modified Sternberg task in which the encoding of memory items, maintenance, and recall were temporally separated. Each trial starts with a fixation period ([-6, -5] s), followed by the stimulus ([-5, -3] s). The stimulus consists of a set of eight consonants at the center of the screen. The middle four, six, or eight letters are the memory items,which determine the set size for the trial (4, 6, or 8, respectively). The outer positions are filled with “X,” which is never a memory item. After the stimulus, the letters disappear from the screen, and the maintenance interval starts ([-3, 0] s).A fixation square is shown throughout fixation, encoding, and maintenance. After maintenance, a probe letter is presented. The subjects respond with a button press to indicate whether the probe was part of the stimulus.The subjects are instructed to respond as rapidly as possible without making errors. The hand used for the response is counterbalanced across subjects within the clinical constraints. After the response, the probe is turned off, and the subjects receive acoustic feedback regarding whether their response was correct or incorrect. Before initiating the next trial, the subjects were encouraged to blink and relax. The subjects perform 50 trials in one session, which last approximately 10 min. Trials with different set sizes are presented in a random order,with the single exception that a trial with an incorrect response is always followed by a trial with a set size of 4."
    task_url = task["Task URL"].values[0]
    return f"Task Name: {task_name}\nTask Description: {task_desc}\nTask URL: {task_url}"

In [76]:
def _get_related_publications(nix: nixio.File) -> List[str]:
    related_publications = nix.sections["General"].sections["Related publications"].props
    names_and_dois = zip(related_publications["Publication name"].values,
                         related_publications["Publication DOI"].values)
    return [f"{name.strip()} ({doi.strip()})" for (name, doi) in names_and_dois]

In [84]:
def write_subject(context: Context):
    subject = context.nix.sections["Subject"].props
    age = subject["Age"].values[0]
    sex = subject["Sex"].values[0]
    context.nwb.subject = Subject(
        subject_id=f"{context.subject:02}",
        age=f"P{int(age)}Y",
        description=_get_subject_description(context),
        species="Homo sapiens",
        sex=_get_sex(sex),
    )

In [78]:
def _get_sex(raw: str) -> str:
    male = ["male", "m", "männlich", "maennlich", "mannlich", "männchen", "mannchen"]
    female = ["female", "f", "weiblich", "weibchen"]
    sex = raw.lower()
    return "M" if sex in male else "F" if sex in female else "O"

In [79]:
def _get_subject_description(context: Context) -> str:
    subject = context.nix.sections["Subject"].props
    handedness = subject["Handedness"].values[0]
    pathology = subject["Pathology"].values[0]
    depth_electrodes = subject["Depth electrodes"].values[0]
    electrodes_in_soz = subject["Electrodes in seizure onset zone (SOZ)"].values[0]
    return f"Handedness: {handedness}\nPathology: {pathology}\nDepth electrodes: {depth_electrodes}\nElectrodes in seizure onset zone (SOZ): {electrodes_in_soz}"


In [80]:
if __name__ == "__main__":
    main()

In [86]:
context = create_context(1, 1)
write_subject(context)
context.nwb

root pynwb.file.NWBFile at 0x139930502805200
Fields:
  file_create_date: [datetime.datetime(2023, 3, 3, 4, 8, 18, 871728, tzinfo=tzlocal())]
  identifier: Human_MTL_units_scalp_EEG_and_iEEG_verbal_WM_subject01_session01
  institution: Universitätsspital Zürich, 8091 Zurich, Switzerland
  lab: Schweizerische Epilepsie-Zentrum, 8008 Zurich, Switzerland
  related_publications: ['Persistent hippocampal neural firing and hippocampal-cortical coupling predict verbal working memory load (doi: 10.1126/sciadv.aav3687)']
  session_description: Task: Sternberg
Description: The task is a modified Sternberg task in which the encoding of memory items, maintenance, and recall were temporally separated. Each trial starts with a fixation period ([-6, -5] s), followed by the stimulus ([-5, -3] s).The stimulus consists of a set of eight consonants at the center of the screen. The middle four, six, or eight letters are the memory items,which determine the set size for the trial (4, 6, or 8, respectively).